# Developing a model from limited data

This notebook presents an example where a new water distribution system network is created from imperfect and missing data. The KY4 network was used as the "perfect" system model; it was then disassembled and the data truncated and skewed to make the "imperfect" data that will be used to create a new model.

In [ ]:
## Initial imports and setup
import geopandas as gpd
import numpy as np
import matplotlib.pylab as plt
import wntr

crs = "EPSG:3547"  # ft

distance_threshold = 100.0

In [ ]:
# TODO, add metadata to hide this cell
wn0 = wntr.network.WaterNetworkModel("../networks/ky4.inp")
wn0.options.time.duration = 24 * 3600
sim = wntr.sim.EpanetSimulator(wn0)
results0 = sim.run_sim()
pressure0 = results0.node["pressure"].loc[24 * 3600, :]
wntr.graphics.plot_network(wn0, node_attribute=pressure0, node_size=30, title="Pressure")
wntr.network.io.write_geojson(wn0, "../data/ky4", crs=crs)

# junctions should have base demand (see PR)
aed0 = wntr.metrics.average_expected_demand(wn0)
print(aed0)
for name, control in wn0.controls():
    print(name, control)
for name, pattern in wn0.patterns():
    print(name, pattern.multipliers)

In [ ]:
# Option 1, direct from GeoJSON files
geojson_files = {
    "junctions": "../data/ky4_junctions.geojson",
    "tanks": "../data/ky4_tanks.geojson",
    "reservoirs": "../data/ky4_reservoirs.geojson",
    "pipes": "../data/ky4_pipes.geojson",
    "pumps": "../data/ky4_pumps.geojson",
}
wn = wntr.network.read_geojson(geojson_files)

In [ ]:
# Option 2, from GeoDataFrames
junctions = gpd.read_file("../data/ky4_junctions.geojson", crs=crs)
tanks = gpd.read_file("../data/ky4_tanks.geojson", crs=crs)
reservoirs = gpd.read_file("../data/ky4_reservoirs.geojson", crs=crs)
pipes = gpd.read_file("../data/ky4_pipes.geojson", crs=crs)
pumps = gpd.read_file("../data/ky4_pumps.geojson", crs=crs)

junctions.set_index("name", inplace=True)
tanks.set_index("name", inplace=True)
reservoirs.set_index("name", inplace=True)
pipes.set_index("name", inplace=True)
pumps.set_index("name", inplace=True)

gis_data = wntr.gis.WaterNetworkGIS(
    {
        "junctions": junctions,
        "tanks": tanks,
        "reservoirs": reservoirs,
        "pipes": pipes,
        "pumps": pumps,
    }
)
wn = wntr.network.from_gis(gis_data)

In [ ]:
pump = wn.get_link("~@Pump-1")
pump.initial_status = "Closed"

In [ ]:
line = "LINK ~@Pump-1 OPEN IF NODE T-3 BELOW  27.6606"  # 90.75 ft
wn.add_control("Pump1_open", line)
line = "LINK ~@Pump-1 CLOSED IF NODE T-3 ABOVE  32.2326"  # 105.75 ft
wn.add_control("Pump1_closed", line)

In [ ]:
# Add patterns
multipliers = [
    0.33,
    0.25,
    0.209,
    0.209,
    0.259,
    0.36,
    0.529,
    0.91,
    1.2,
    1.299,
    1.34,
    1.34,
    1.32,
    1.269,
    1.25,
    1.25,
    1.279,
    1.37,
    1.519,
    1.7,
    1.75,
    1.669,
    0.899,
    0.479,
]
default_pattern_name = wn.options.hydraulic.pattern
wn.add_pattern(default_pattern_name, multipliers)

In [ ]:
aed = wntr.metrics.average_expected_demand(wn)
print(aed)

# Run simulation, plot_results
# TODO, this shoudl be 24, but the simulation doesn't run to completion, due to demands?
wn.options.time.duration = 0  # 24*3600
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()
pressure = results.node["pressure"].loc[0, :]
wntr.graphics.plot_network(wn, node_attribute=pressure, node_size=30, title="Pressure")

### Create a model from imperfect geospatial data

In [ ]:
# Load pipe data
# Missing junctions (no start and end node names or locations)
# Pipe endpoints are not aligned
diconnected_pipes = gpd.read_file("../data/ky4_disconnected_pipes.geojson")
# pipes.set_index('index', inplace=True)
# del pipes['start_node']
# del pipes['end_node_n']
pipesA, junctionsA = wntr.gis.geospatial.reconnect_network(diconnected_pipes, distance_threshold)

In [ ]:
fig, ax = plt.subplots()
diconnected_pipes.plot(color="b", linewidth=4, ax=ax)
pipesA.plot(color="r", linewidth=2, ax=ax)
junctionsA.plot(color="k", ax=ax)

In [ ]:
junctionsB = gpd.GeoDataFrame(junctionsA, columns=["geometry"])
junctionsB["base_demand"] = 0
junctionsB["elevation"] = 0
pipesB = pipesA.rename(columns={"new_start_node": "start_node_name", "new_end_node": "end_node_name"}).set_index(
    "index"
)
junctionsB

In [ ]:
# build the wn_gis object
gis_data = wntr.gis.WaterNetworkGIS(
    {
        "junctions": junctionsB,
        #'tanks': tanks,
        #'reservoirs': reservoirs,
        "pipes": pipesB,
        #'pumps': pumps
    }
)
wn = wntr.network.from_gis(gis_data)  # TODO this won't work until PR 452 is merged
wntr.graphics.plot_network(wn, node_attribute=pressure, node_size=30, title="elevation")

In [ ]:
# TODO, recreate this file using complete column names and no start or end node names
# TODO rename the file 'ky4_diconnected_pipes.geojson'
# TODO rename 'index' to 'name'
# TODO same crs as above
diconnected_pipes = gpd.read_file("../data/ky4_disconnected_pipes.geojson", crs=crs)
diconnected_pipes.set_index("index", inplace=True)

pipes, junctions = wntr.gis.geospatial.connect_lines(diconnected_pipes, distance_threshold)

In [ ]:
fig, ax = plt.subplots()
diconnected_pipes.plot(color="b", linewidth=4, ax=ax)
pipes.plot(color="r", linewidth=2, ax=ax)
junctions.plot(color="k", ax=ax)

# The following lines should not be needed, see PR
junctions["node_type"] = "Junction"
pipes["link_type"] = "Pipe"

# Assign elevation to junctions - using raster
# TODO
junctions["elevation"] = 100

# Load reservoir - complete dataset, location and head
reservoirs = gpd.read_file("../data/ky4_reservoirs.geojson", crs=crs)
reservoirs.set_index("name", inplace=True)
# TODO snap reservoirs to nearest junction
# snap_reservoirs = wntr.gis.snap(reservoirs, junctions, distance_threshold)

# Load tank data - complete dataset, location, min/max level, diameter
tanks = gpd.read_file("../data/ky4_tanks.geojson", crs=crs)
tanks.set_index("name", inplace=True)
# TODO snap tanks to nearest junction

# Load pump data - location, settings, start, end node)
pumps = gpd.read_file("../data/ky4_pumps.geojson", crs=crs)
pumps.set_index("name", inplace=True)
# TODO snap end points to nearest junction, add pump and close bipass?


# build the wn_gis object
# TODO, add back in tanks, reservoirs, and pumps
gis_data = wntr.gis.WaterNetworkGIS(
    {
        "junctions": junctions,
        #'tanks': tanks,
        #'reservoirs': reservoirs,
        "pipes": pipes,
        #'pumps': pumps
    }
)
wn = wntr.network.from_gis(gis_data)  # TODO this won't work until PR 452 is merged

In [ ]:
# Add controls, initial status, and pattern (same code as above)

# Add demands, estimated from building size
buildings = gpd.read_file("../data/ky4_buildings.geojson", crs=crs)
buildings.to_crs(crs, inplace=True)
buildings["area"] = buildings.area
buildings["base_demand"] = buildings["area"] / 0.00001

buildings.geometry = buildings.geometry.centroid
snap_buildings = wntr.gis.snap(buildings, junctions, distance_threshold)
buildings["junction"] = None
buildings.loc[snap_buildings.index, "junction"] = snap_buildings.loc[:, "node"]

In [ ]:
category = None
pattern_name = "1"
print(buildings)
for i, row in buildings.iterrows():
    junction_name = buildings.loc[i, "junction"]
    if junction_name is None:
        continue
    base_demand = buildings.loc[i, "base_demand"]
    junction = wn.get_node(junction_name)
    junction.demand_timeseries_list.append((base_demand, pattern_name, category))
wntr.graphics.plot_network(wn, node_attribute=pressure, node_size=30, title="elevation")
# Run simulation, plot_results
wn.options.time.duration = 24 * 3600
sim = wntr.sim.EpanetSimulator(wn)
results = sim.run_sim()
pressure = results.node["pressure"].loc[24 * 3600, :]